In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
def make_pattern(line_segment,segment_length, test = False):

    beta = 0 
    x_start, x_end= line_segment[0][0], line_segment[-1][0]
    y_start, y_end = line_segment[0][1], line_segment[-1][1]

    final_array = np.zeros((9,2))

    #horisontal
    if  y_end == y_start and x_start != x_end and [x_end, y_end] != [0,0]:
        
        check = x_end - x_start 
        if check < 0: 
            beta = 1
    
        x_array = np.linspace(x_start,x_start + (-1)**beta * segment_length,5)
        y_array = np.array([y_start,y_start - (-1)**beta * segment_length/4, y_start + (-1)**beta * segment_length/4])

        final_array = np.array([[x_array[0],y_array[0]],
                                [x_array[1],y_array[0]],
                                [x_array[1],y_array[1]],
                                [x_array[2],y_array[1]],
                                [x_array[2],y_array[0]],
                                [x_array[2],y_array[2]],
                                [x_array[3],y_array[2]],
                                [x_array[3],y_array[0]],
                                [x_array[4],y_array[0]]])
    
    # vertical
    elif x_start == x_end and y_end != y_start:
       
        check = y_end - y_start
        if check < 0:
            beta = 1

        y_array = np.linspace(y_start,y_start + (-1)**beta * segment_length,5)
        x_array = np.array([x_start,x_start + (-1)**beta * segment_length/4, x_start - (-1)**beta * segment_length/4])

        final_array = np.array([[x_array[0],y_array[0]],
                                [x_array[0],y_array[1]],
                                [x_array[1],y_array[1]],
                                [x_array[1],y_array[2]],
                                [x_array[0],y_array[2]],
                                [x_array[2],y_array[2]],
                                [x_array[2],y_array[3]],
                                [x_array[0],y_array[3]],
                                [x_array[0],y_array[4]]])

    #To make sure that [0,0] is not added twice in the last side of the square
    if [x_start, y_start] == [0,1] and [x_end, y_end] == [0,0] and segment_length < 1:
        final_array = np.zeros((9,2))
    
    return final_array


def line_coordinates(line_segment,N,L):
    '''
    Function that helps constructing the fractal.
    '''
    
    if N == 0:
        line_lenght = 1
    else:
        line_lenght = 7*8**(N)
    
    return_coord = np.zeros((line_lenght,2))
    
    if N != 0:
        
        return_coord[0:9] = make_pattern(line_segment,L)

        for n in range(N-1):
            
            copied_array = np.copy(return_coord)
                
            ink = 0
            for i in range(9*8**n):
                pattern = make_pattern(copied_array[i:i+2],L/(4**(n+1)))
                #print(copied_array[i:i+2])

                zero_elem = 0

                for k in range(len(pattern)):
                    if pattern[k,0] == 0 and pattern[k,1] == 0:
                        zero_elem += 1
                    
                if zero_elem < 3:
                    return_coord[9*ink:9*(ink+1)] = pattern
                    ink += 1
    else:
        return_coord = line_segment[0]

    return return_coord


def fractal_generator(N,L):
    '''
    Parameters:
        N = Number of l's
        L = Length of sides
    Output:
        fractal = Array containing all points (x,y) in fractal
    '''
    
    if N == 0:
        line_lenght = 1
    else:
        line_lenght = 7*8**(N)

    fractal = np.zeros((4*line_lenght+1,2))
    
    #Each side in the square:
    fractal[0:line_lenght] = line_coordinates(np.array([[0,0],[L,0]]),N,L)
    fractal[line_lenght:2*line_lenght] = line_coordinates(np.array([[L,0],[L,L]]),N,L)
    fractal[2*line_lenght:3*line_lenght] = line_coordinates(np.array([[L,L],[0,L]]), N,L)
    fractal[3*line_lenght:4*line_lenght] = line_coordinates(np.array([[0,L],[0,0]]),N,L)
    
    list_of_zeros = []

    for i in range(1,len(fractal)-1):
        if fractal[i,0] == 0 and fractal[i,1] == 0:
            list_of_zeros.append(i)

    fractal = np.delete(fractal, list_of_zeros, axis = 0)

    return fractal

In [ ]:
def find_corners(fractal_array):
    '''
    function that finds the corners in the fractal.
    '''

    #Bruke np.max og np.min?

    biggestx = np.max(fractal_array[:,0])
    biggesty = np.max(fractal_array[:,1])   
    smallestx = np.min(fractal_array[:,0])
    smallesty = np.min(fractal_array[:,1])

    return biggestx,biggesty,smallestx,smallesty


def make_lattice(fractal_array):
    '''
    parameters: 
        N = the order of the fractal
        fractal_array = array of fractal points
    '''
    #must find the corners of the fractal! 
    bigX, bigY, smallX, smallY = find_corners(fractal_array)
    
    #making the lattice
    lattice_constant = fractal_array[1][0] - fractal_array[0][0]
    startX = smallX - lattice_constant*2
    stopX = bigX + lattice_constant*2
    startY = smallY - lattice_constant*2
    stopY = bigY + lattice_constant*2
    
    x = np.arange(startX, stopX, lattice_constant)
    y = np.arange(startY, stopY, lattice_constant)

    return x,y